# Playground for cause and effect two sentences

TODO:explain 

In [1]:
import os
import numpy as np
import openai
import pandas as pd
from tqdm import tqdm
from setups.ce_two_sentences_setup import CETwoSentences

# Load your API key from an environment variable or secret management service
openai.api_key = os.getenv("OPENAI_KEY")

In [2]:
gpt3_model = "text-davinci-001"
#gpt3_model = "text-curie-001"
#gpt3_model = "text-babbage-001"
#gpt3_model = "text-ada-001"

In [3]:
# test run
def gpt3_response(prompt, model):
    response = openai.Completion.create(engine=model, prompt=prompt, max_tokens=20, temperature=0)
    return(response["choices"][0]["text"])

In [4]:
ce_2_sentences = CETwoSentences()

In [5]:
ce_2_sentences.generate_sequences_df()

,sequence,answer_cause,answer_effect,switched
0,The child hurt their knee. The child started c...,The child hurt their knee.,The child started crying.,False
1,The child started crying. The child hurt their...,The child hurt their knee.,The child started crying.,True
2,My car got dirty. I washed the car.,My car got dirty.,I washed the car.,False
3,I washed the car. My car got dirty.,My car got dirty.,I washed the car.,True
4,Someone fainted. Someone called 911.,Someone fainted.,Someone called 911.,False
...,...,...,...,...
97,Ted had a hangover. Ted went out drinking.,Ted went out drinking.,Ted had a hangover.,True
98,Alice ate the last cookie. Bob opened the jar ...,Alice ate the last cookie.,Bob opened the jar and was frustrated.,False
99,Bob opened the jar and was frustrated. Alice a...,Alice ate the last cookie.,Bob opened the jar and was frustrated.,True
100,Jimmy took the last piece of cake. Mom rinsed ...,Jimmy took the last piece of cake.,Mom rinsed the tray.,False


In [6]:
ce_2_sentences.save_sequences_df(filepath="data/bigbench_csvs/ce_two_sentences.csv")

### Zero shot

In [7]:
prompt_zs_cause = ce_2_sentences.generate_single_prompt_zero_shot(n_subset=0, switched=False, question="cause")
prompt_zs_effect = ce_2_sentences.generate_single_prompt_zero_shot(n_subset=0, switched=False, question="effect")

prompt_zs_cause_switched = ce_2_sentences.generate_single_prompt_zero_shot(n_subset=0, switched=True, question="cause")
prompt_zs_effect_switched = ce_2_sentences.generate_single_prompt_zero_shot(n_subset=0, switched=True, question="effect")

print(prompt_zs_cause + gpt3_response(prompt_zs_cause, gpt3_model) + "\n")
print(prompt_zs_effect + gpt3_response(prompt_zs_effect, gpt3_model) + "\n")
print(prompt_zs_cause_switched + gpt3_response(prompt_zs_cause_switched, gpt3_model) + "\n")
print(prompt_zs_effect_switched + gpt3_response(prompt_zs_effect_switched, gpt3_model) + "\n")

The child hurt their knee. The child started crying. Question: Which sentence caused the other? Answer by copying the sentence: 

The child started crying.

The child hurt their knee. The child started crying. Question: Which sentence is the effect of the other? Answer by copying the sentence: 

The child's crying caused them to hurt their knee.

The child started crying. The child hurt their knee. Question: Which sentence caused the other? Answer by copying the sentence: 

The child started crying.

The child hurt their knee.

The child started crying. The child hurt their knee. Question: Which sentence is the effect of the other? Answer by copying the sentence: 

The child started crying because they hurt their knee.



In [8]:
print(ce_2_sentences.generate_all_prompts_zero_shot(question='cause')[:3])
print(ce_2_sentences.generate_all_prompts_zero_shot(question='effect')[:3])

['The child hurt their knee. The child started crying. Question: Which sentence caused the other? Answer by copying the sentence: ', 'The child started crying. The child hurt their knee. Question: Which sentence caused the other? Answer by copying the sentence: ', 'My car got dirty. I washed the car. Question: Which sentence caused the other? Answer by copying the sentence: ']
['The child hurt their knee. The child started crying. Question: Which sentence is the effect of the other? Answer by copying the sentence: ', 'The child started crying. The child hurt their knee. Question: Which sentence is the effect of the other? Answer by copying the sentence: ', 'My car got dirty. I washed the car. Question: Which sentence is the effect of the other? Answer by copying the sentence: ']


### One shot

In [9]:
prompt_os_cause = ce_2_sentences.generate_single_prompt_one_shot(n_subset=0, switched=False, question="cause")
prompt_os_effect = ce_2_sentences.generate_single_prompt_one_shot(n_subset=0, switched=False, question="effect")

prompt_os_cause_switched = ce_2_sentences.generate_single_prompt_one_shot(n_subset=0, switched=True, question="cause")
prompt_os_effect_switched = ce_2_sentences.generate_single_prompt_one_shot(n_subset=0, switched=True, question="effect")

print(prompt_os_cause + gpt3_response(prompt_os_cause, gpt3_model) + "\n")
print(prompt_os_effect + gpt3_response(prompt_os_effect, gpt3_model) + "\n")

print(prompt_os_cause_switched + gpt3_response(prompt_os_cause_switched, gpt3_model) + "\n")
print(prompt_os_effect_switched + gpt3_response(prompt_os_effect_switched, gpt3_model) + "\n")

The bottle was left open. The soda went flat. Question: Which sentence caused the other? Answer by copying the sentence: The bottle was left open.
The child hurt their knee. The child started crying. Question: Which sentence caused the other? Answer by copying the sentence: 

The child hurt their knee.

The man got lost in the woods. A search party was assembled. Question: Which sentence is the effect of the other? Answer by copying the sentence: A search party was assembled.
The child hurt their knee. The child started crying. Question: Which sentence is the effect of the other? Answer by copying the sentence: 

The child started crying.

A search party was assembled. The man got lost in the woods. Question: Which sentence caused the other? Answer by copying the sentence: The man got lost in the woods.
The child started crying. The child hurt their knee. Question: Which sentence caused the other? Answer by copying the sentence: 

The child hurt their knee.

A flower grew. The gardener

In [10]:
print(ce_2_sentences.generate_all_prompts_one_shot(question='cause')[:3])
print("\n")
print(ce_2_sentences.generate_all_prompts_one_shot(question='effect')[:3])
print("\n")

['The milk was left outside the fridge over night. The man sipped the milk and spit it out. Question: Which sentence caused the other? Answer by copying the sentence: The milk was left outside the fridge over night.\nThe child hurt their knee. The child started crying. Question: Which sentence caused the other? Answer by copying the sentence: ', 'A search party was assembled. The man got lost in the woods. Question: Which sentence caused the other? Answer by copying the sentence: The man got lost in the woods.\nThe child started crying. The child hurt their knee. Question: Which sentence caused the other? Answer by copying the sentence: ', 'The laptop ran out of battery. George plugged the laptop charger in. Question: Which sentence caused the other? Answer by copying the sentence: The laptop ran out of battery.\nMy car got dirty. I washed the car. Question: Which sentence caused the other? Answer by copying the sentence: ']


['The girl skipped school. The girl got detention. Question

### K=5 shot

In [11]:
prompt_ks_cause = ce_2_sentences.generate_single_prompt_k_shot(n_subset=0, k=5, switched=False, question="cause")
prompt_ks_effect = ce_2_sentences.generate_single_prompt_k_shot(n_subset=0, k=5, switched=False, question="effect")

prompt_ks_cause_switched = ce_2_sentences.generate_single_prompt_k_shot(n_subset=0, k=5, switched=True, question="cause")
prompt_ks_effect_switched = ce_2_sentences.generate_single_prompt_k_shot(n_subset=0, k=5, switched=True, question="effect")

print(prompt_ks_cause + gpt3_response(prompt_ks_cause, gpt3_model) + "\n")
print(prompt_ks_effect + gpt3_response(prompt_ks_effect, gpt3_model) + "\n")
print(prompt_ks_cause_switched + gpt3_response(prompt_ks_cause_switched, gpt3_model) + "\n")
print(prompt_ks_effect_switched + gpt3_response(prompt_ks_effect_switched, gpt3_model) + "\n")

The floor was wet. The man sued. Question: Which sentence caused the other? Answer by copying the sentence: The floor was wet.
The company's posted strong earnings. The company's stock went up. Question: Which sentence caused the other? Answer by copying the sentence: The company's posted strong earnings.
School was on break. The family went on vacation. Question: Which sentence caused the other? Answer by copying the sentence: School was on break.
Jill won the lawsuit against Acme. Acme paid Jill's legal fees. Question: Which sentence caused the other? Answer by copying the sentence: Jill won the lawsuit against Acme.
Alice didn't wake up. Alice missed her flight. Question: Which sentence caused the other? Answer by copying the sentence: Alice didn't wake up.
The child hurt their knee. The child started crying. Question: Which sentence caused the other? Answer by copying the sentence: 
The child hurt their knee.

The basketball player was shoved. The basketball player took a free thro

In [12]:
print(ce_2_sentences.generate_all_prompts_k_shot(k=5, question='cause')[:3])
print("\n")
print(ce_2_sentences.generate_all_prompts_k_shot(k=5, question='effect')[:3])

["Alice ate the last cookie. Bob opened the jar and was frustrated. Question: Which sentence caused the other? Answer by copying the sentence: Alice ate the last cookie.\nAlice didn't wake up. Alice missed her flight. Question: Which sentence caused the other? Answer by copying the sentence: Alice didn't wake up.\nThe driver ran a red light. The driver was pulled over and was fined. Question: Which sentence caused the other? Answer by copying the sentence: The driver ran a red light.\nThe power was out for days. The meat spoiled. Question: Which sentence caused the other? Answer by copying the sentence: The power was out for days.\nA watermelon exploded. A man was covered in sticky red goo. Question: Which sentence caused the other? Answer by copying the sentence: A watermelon exploded.\nThe child hurt their knee. The child started crying. Question: Which sentence caused the other? Answer by copying the sentence: ", "The man sued. The floor was wet. Question: Which sentence caused the 